# 資料預處理

載入所需模組

In [102]:
import tensorflow as tf
import numpy as np
import gc
import matplotlib.pyplot as plt
import pandas as pd
import time
from PIL import Image

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Flatten, Dense
from keras.models import Sequential, Model

from keras.preprocessing import image
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

from keras.applications.xception import preprocess_input
from keras.applications.xception import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2 
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50

建立train資料夾及test資料夾路徑

In [103]:
train_path = "./AIA_imageClassification/Data/data/examples/may_the_4_be_with_u/where_am_i/train"
test_path = "./AIA_imageClassification/Data/data/examples/may_the_4_be_with_u/where_am_i/test"

## Step.1 利用 ImageDataGenerator 圖片增益

ImageDataGenerator : 生成批次的帶實時數據增益的張量圖像數據，數據將按批次無限循環。

ImageDataGenerator : (參數說明)

1. preprocessing_function: 

    應用於每個輸入的函數，這個函數會在任何其他改變之前運行。這個函數需要一個參數：一張圖像（為 3 的 Numpy 張量），並且應該輸出一個同尺寸的 Numpy 張量。

2. rotation_range:

    整數。隨機旋轉的度數範圍。

3. width_shift_range:

    (總寬度的比例）。隨機水平移動的範圍。

4. height_shift_range:

    （總高度的比例）。隨機垂直移動的範圍。

5. shear_range:

    浮點數。剪切強度（以弧度逆時針方向剪切角度）。

6. zoom_range:

    浮點數 或 [lower, upper]。隨機縮放範圍。如果是浮點數，[lower, upper] = [1-zoom_range, 1+zoom_range]。

7. horizontal_flip:

    布爾值。隨機水平翻轉。

In [104]:
datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input,
                                    #rotation_range=30,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True)

超參數設定

In [105]:
batch_size = 32
image_size = (250,250)
total_traindata_length = 2985 

## Step.2 利用 flow_from_directory 實時讀取圖片

flow_from_directory : 以目錄路徑為參數，生成批次的 增益的/標準化的 數據。在生成的批次數據上無限制地無限次循環。

flow_from_directory :(參數說明)

1. target_size: 

    整數元組 (height, width)，默認：(256, 256)。所有的圖像將被調整到的尺寸。
2. batch_size: 
    
    一批數據的大小（默認 32）。

In [106]:
# 從資料夾中讀取圖片
train_generator = datagen.flow_from_directory(
        train_path,
        target_size=image_size,
        batch_size=batch_size,
        )

Found 2985 images belonging to 15 classes.


使用class_indices屬性生成生成器生成的每個類的索引

In [107]:
print(train_generator.class_indices)

{'00_kitchen': 0, '01_street': 1, '02_industrial': 2, '03_insidecity': 3, '04_forest': 4, '05_livingroom': 5, '06_opencountry': 6, '07_PARoffice': 7, '08_mountain': 8, '09_CALsuburb': 9, '10_coast': 10, '11_store': 11, '12_bedroom': 12, '13_tallbuilding': 13, '14_highway': 14}


# 建立模型

### 採用遷移式學習

遷移式學習 : 為了偷懶，在訓練好了的模型上接著訓練其他內容，充分使用原模型的理解力

![TransferLearning](./AIA_imageClassification/photo/TransferLearning.PNG)

(圖片擷取自:[莫煩python](https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/5-16-transfer-learning/))

遷移式學習參考資料:
    
   1.[莫煩python](https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/5-16-transfer-learning/)
    
   2.[多重預訓練視覺模型的遷移學習](https://itw01.com/Q23KE77.html)
    
   3.[一文看懂迁移学习：怎样用预训练模型搞定深度神经网络？](https://zhuanlan.zhihu.com/p/27657264)

## Step.1採用模型 : ResNet50

![TransferLearning](./AIA_imageClassification/photo/ResNet50.PNG)

(圖片擷取自:[使用Tensorflow实现残差网络ResNet-50](https://blog.csdn.net/liangyihuai/article/details/79140481))

Resnet整個網絡的成功點就是在於應用了殘差塊，其實現也非常簡單。原始輸入xx經過一個conv-relu-conv組合層，輸出一個F(x)F(x)，然後把這個輸出和原始輸出做一個加法(elewise-add)，即H(x)=F(x) +xH(x)=F(x)+x，如下圖所示。所以殘差塊和原始的CNN網絡唯一的區別就是原始輸入經過一個identity激活函數（叫做恆等映射，即y=xy=x）疊加到卷積輸出上，這個支路叫做shortcut連接。

![TransferLearning](./AIA_imageClassification/photo/殘差塊.PNG)

(圖片擷取自:[深度学习笔记（5）——学术界的霸主Resnet](https://blog.csdn.net/qq_21190081/article/details/75933329))

ResNet50參考資料

1.[深度学习笔记（5）——学术界的霸主Resnet](https://blog.csdn.net/qq_21190081/article/details/75933329)

2.[使用Tensorflow实现残差网络ResNet-50](https://blog.csdn.net/liangyihuai/article/details/79140481)

3.[无需数学背景，读懂 ResNet、Inception 和 Xception 三大变革性架构](https://www.jiqizhixin.com/articles/2017-08-19-4)

4.[Deep Residual Networks（ResNet） 简介](https://blog.csdn.net/sxf1061926959/article/details/54973588)

5.[搭积木般构建深度学习网络 —— ResNet50完整代码解析](https://blog.csdn.net/wangli0519/article/details/73744595)

6.[Keras 中文文档](https://keras.io/zh/applications/)

ResNet50 :(參數說明)

1. weights:

    None 代表隨機初始化， 'imagenet' 代表加載在 ImageNet 上預訓練的權值
 
2. include_top: 

    是否包括頂層的全連接層。
    
3. input_shape: 

    可選，輸入尺寸元組，僅當include_top=False 時有效（不然輸入形狀必須是(224, 224, 3) （channels_last 格式）或(3, 224, 224) （channels_first 格式），因為預訓練模型是以這個大小訓練的）。輸入尺寸必須是三個數字，且寬高必須不小於 197，比如 (200, 200, 3) 是一個合法的輸入尺寸。

In [108]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(250,250,3))
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 250, 250, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 125, 125, 64) 9472        input_5[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 125, 125, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_197 (Activation)     (None, 125, 125, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

## Step.2 建立全連接層

In [109]:
x = base_model.output
x = GlobalAveragePooling2D()(x) # GlobalAveragePooling2D 将 MxNxC 的张量转换成 1xC 张量，C是通道数
x = Dense(1024,activation='relu')(x)
predictions = Dense(15,activation='softmax')(x)
model = Model(inputs=base_model.input,outputs=predictions)

In [110]:
#base_model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

In [111]:
#model.summary()

## Step.3 微調ResNet50

1.
通常的做法是截斷預訓練網絡的最後一層（softmax層），並將其替換為與我們自己的問題相關的新softmax層。例如，ImageNet上經過預先訓練的網絡帶有1000個類別的softmax層。

2.
使用較小的學習速率來訓練網絡。由於我們期望預先訓練的權重與隨機初始化權重相比已經非常好，我們不希望過快和過多地扭曲它們。通常的做法是使初始學習率比用於划痕訓練的初始學習率小10倍。

3.
凍結預訓練網絡的前幾層的權重也是一種常見做法。這是因為前幾層捕獲了與我們的新問題相關的曲線和邊緣等通用特徵。我們希望保持這些重量不變。相反，我們將讓網絡專注於學習後續層中的數據集特定功能

![TransferLearning](./AIA_imageClassification/photo/FineTune.PNG)

(圖片擷取自:[How to use transfer learning and fine-tuning in Keras and Tensorflow to build an image recognition system and classify (almost) any object](https://deeplearningsandbox.com/how-to-use-transfer-learning-and-fine-tuning-in-keras-and-tensorflow-to-build-an-image-recognition-94b0b02444f2))

Fine-tuning 參考資料

1.[How to use transfer learning and fine-tuning in Keras and Tensorflow to build an image recognition system and classify (almost) any object](https://deeplearningsandbox.com/how-to-use-transfer-learning-and-fine-tuning-in-keras-and-tensorflow-to-build-an-image-recognition-94b0b02444f2)

2.[A Comprehensive guide to Fine-tuning Deep Learning Models in Keras (Part I)](https://flyyufelix.github.io/2016/10/03/fine-tuning-in-keras-part1.html)

3.[A Comprehensive guide to Fine-tuning Deep Learning Models in Keras (Part II)](https://flyyufelix.github.io/2016/10/08/fine-tuning-in-keras-part2.html)


In [112]:
from keras.optimizers import Adagrad
from keras.optimizers import SGD
def setup_to_fine_tune(model,base_model):
    
    layer_num = len(model.layers)
    for layer in model.layers[:int(layer_num * 0.9)]:
        layer.trainable = False

    for layer in model.layers[int(layer_num * 0.9):]:
        layer.trainable = True
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [113]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

load_weights(參數說明)

1.model.load_weights(filepath, by_name=False) :

從 HDF5 文件（由 save_weights 創建）中加載權重。默認情況下，模型的結構應該是不變的。如果想將權重載入不同的模型（部分層相同）， 設置 by_name=True 來載入那些名字相同的層的權重。

2.model.save_weights（filepath）:

將模型權重存儲為HDF5文件。

In [114]:
try:
    model.load_weights("./AIA_imageClassification/Model/ResNet50_Model_good123.h5")
    print("載入模型成功")
except:
    print("載入模型失敗")

載入模型失敗


超參數設定

In [115]:
epochs = 200

# 通常這個設定為總資料的數量除以批次大小
steps_per_epoch = total_traindata_length / batch_size

# 模型訓練

## Step.1 使用 EarlyStopping 設定訓練停損點

EarlyStopping : 當被監測的數量不再提升，則停止訓練。

EarlyStopping :(參數說明)

1.patience: 沒有進步的訓練輪數，在這之後訓練就會被停止。

2.monitor: 被監測的數據。

EarlyStopping 參考資料:

1.[Keras中文文本](https://keras.io/zh/callbacks/)

## Step.2 利用fit_generator最小化顯存占用比率/數據Batch化

fit_generator : 使用 Python 生成器逐批生成的數據，按批次訓練模型。生成器與模型並行運行，以提高效率。例如，這可以讓你在 CPU 上對圖像進行實時數據增強，以在 GPU 上訓練模型。

fit_generator :(參數說明)
    
1.generator:
    
一個 (inputs, targets, sample_weights) 元組。所有的數組都必須包含同樣數量的樣本。生成器將無限地在數據集上循環。當運行到第 steps_per_epoch 時，記一個 epoch 結束。

2.steps_per_epoch:

在聲明一個 epoch 完成並開始下一個 epoch 之前從 generator 產生的總步數（批次樣本）。它通常應該等於你的數據集的樣本數量除以批量大小。可選參數 Sequence：如果未指定，將使用len(generator) 作為步數。

3.epochs:

整數，數據的迭代總輪數。請注意，與 initial_epoch 一起，參數 epochs 應被理解為 「最終輪數」。模型並不是訓練了 epochs 輪，而是到第 epochs 輪停止訓練。

4.class_weight:

將類別映射為權重的字典。

5.callbacks: 

在訓練時調用的一系列回調函數。

參考資料:
    
1.[Keras中文文本](https://keras-cn.readthedocs.io/en/latest/models/model/)

In [116]:
#setup_to_fine_tune(model,base_model)

early_stopping = EarlyStopping(patience=4,monitor='loss')
history_ft = model.fit_generator(generator=train_generator,
                                 steps_per_epoch=steps_per_epoch,
                                 epochs=epochs,
                                 class_weight='auto',
                                callbacks=[early_stopping])

Epoch 1/200
94/93 [==============================] - 33s 352ms/step - loss: 2.4897 - acc: 0.2067
Epoch 2/200
94/93 [==============================] - 29s 311ms/step - loss: 1.7585 - acc: 0.5219
Epoch 3/200
94/93 [==============================] - 29s 309ms/step - loss: 1.3018 - acc: 0.6941
Epoch 4/200
94/93 [==============================] - 29s 308ms/step - loss: 0.9916 - acc: 0.7747
Epoch 5/200
94/93 [==============================] - 28s 302ms/step - loss: 0.7533 - acc: 0.8352
Epoch 6/200
94/93 [==============================] - 31s 329ms/step - loss: 0.6398 - acc: 0.8490
Epoch 7/200
94/93 [==============================] - 28s 302ms/step - loss: 0.5479 - acc: 0.8670
Epoch 8/200
94/93 [==============================] - 30s 320ms/step - loss: 0.4771 - acc: 0.8775
Epoch 9/200
94/93 [==============================] - 30s 315ms/step - loss: 0.4218 - acc: 0.8971
Epoch 10/200
94/93 [==============================] - 29s 313ms/step - loss: 0.3817 - acc: 0.9027
Epoch 11/200
94/93 [=========

## Step.3儲存模型參數

In [117]:
model.save_weights("./AIA_imageClassification/Model/ResNet50_Model_good123.h5")
print("Saved model")

Saved model


# 針對 test 資料集進行預測

## Step.1 使用 flow_from_directory 提取test資料夾

In [118]:
test_batch_size = 32

steps = 1500 / test_batch_size

test_generator = datagen.flow_from_directory(
        test_path,
        target_size=image_size,
        shuffle=False,
        class_mode=None,
        batch_size=test_batch_size)

Found 1500 images belonging to 1 classes.


## Step.2 使用 predict_generator 預測 test 資料

predict_generator : 為來自數據生成器的輸入樣本生成預測。

predict_generator :(參數說明)

1.generator:
    
返回批量輸入樣本的生成器

2.steps:

在停止之前，來自 generator 的總步數 (樣本批次)。可選參數 Sequence：如果未指定，將使用len(generator) 作為步數。

In [119]:
y_predict = model.predict_generator(test_generator, steps=steps)

檢驗test資料形狀是否與未預測前相同

In [120]:
y_predict.shape

(1500, 15)

# 將預測提交至Kaggle

## Step.1 抓取test資料之檔案名稱

In [121]:
test_filenames = test_generator.filenames

## Step.2 刪除filenames多餘字串

In [122]:
test_filenames = [item[8:] for item in (test_filenames)]
test_filenames = [item[:-4] for item in (test_filenames)]

## Step.3 抓取每一筆資料最大值

In [123]:
probability_to_label = [np.argmax(i) for i in (y_predict)]
probability_to_label[0]

7

## Step.4 將所需資料存至DataFrame

In [124]:
data = {'id': test_filenames,'class': probability_to_label }

predicted_dataframe = pd.DataFrame(data=data)
predicted_dataframe.head()

## Step.5 儲存成csv檔，提交至Kaggle

In [126]:
predicted_dataframe.to_csv('./AIA_imageClassification/Submission/ResNet50_0715_3.csv', index=False)

![](./AIA_imageClassification/photo/kaggle.PNG)

# 遇到問題

### Q1

Resource exhausted: OOM when allocating tensor with shape[2304,384] Traceback (most recent call last):

解決方法:
    
https://github.com/tensorflow/tensorflow/issues/136

===========================================================================================

### Q2

.ipynb_checkpoints 存在在training dataset中

導致datagen讀取images時classes增加一個類別

解決方法:

開啟terminal，進入到train資料夾當中，並且下指令 re -r .ipynb_checkpoints(刪除checkpoints)

參考解決方式:
    
http://forums.fast.ai/t/how-to-remove-ipynb-checkpoint/8532/27

# ResNet50 模型參考

## 參考資料

https://github.com/sebastianbk/finetuned-resnet50-keras/blob/master/resnet50_train.py

https://github.com/keras-team/keras/issues/7177